In [1]:
#!pip3 install pandas
#!pip3 install numpy
#!pip3 install scipy

In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import itertools

Reading identity-based clusters and WSSD gene-family copy numbers:

In [3]:
wssd=pd.read_table("data/SD98_WSSD.tsv", sep="\t", header=0, index_col=0)

In [4]:
clusters=[]
with open("data/SD98_exon_clusters.txt") as f:
    for index, line in enumerate(f):
        line_strip=line.strip()
        line_split=line_strip.split(",")
        clusters.append(line_split)

To address dispersion of copy number within a cluster we used the median absolute deviation (MAD) metric. We defined a function to get MAD scores per cluster:

In [5]:
def get_mad(elements):
    wssd_clust=wssd[wssd.index.isin(elements)]
    wssd_clust_median=wssd_clust.median(axis=1)
    return stats.median_abs_deviation(wssd_clust_median)

We selected pairwise clusters based on shared exons only if genes had similar overall WSSD copy number:

In [8]:
low_dispersion_clusters=list()
high_dispersion_clusters=list()

for cluster in clusters:
    if get_mad(cluster) < 1:
        low_dispersion_clusters.append(cluster)
    else:
        high_dispersion_clusters.append(cluster)
           
with open('results/low_dispersion_clusters.txt', 'w') as f:
    for cluster in low_dispersion_clusters:
        f.write(",".join(sorted(str(c) for c in cluster)) + '\n')

with open('results/high_dispersion_clusters.txt', 'w') as f:
    for cluster in high_dispersion_clusters:
        f.write(",".join(sorted(str(c) for c in cluster)) + '\n')   

Then, we generated gene families based on shared protein coding genes or unprocessed pseudogenes:

In [9]:
genes=[]

for cluster in low_dispersion_clusters:
    for element in cluster:
        if any(x in element for x in ["protein_coding","unprocessed_pseudogene"]):
            genes.append(element)

genes=list(set(genes))
len(genes)

1673

In [10]:
families=list()

for gene in genes:   
    gene_cluster = list()
    gene_cluster.append(gene)
    
    i=0
    while True:
        if any(x in gene_cluster[i] for x in ["protein_coding","unprocessed_pseudogene"]): 
            for cluster in low_dispersion_clusters:
                if gene_cluster[i] in cluster:
                    gene_cluster = list(set(gene_cluster+cluster))
                    
        i+=1
        if i == len(gene_cluster):
            families.append(sorted(str(c) for c in gene_cluster))
            break

In [12]:
families.sort()
families_dedup=list(k for k,_ in itertools.groupby(families))

with open('results/low_dispersion_families.txt', 'w') as f:
    for family in families_dedup:
        f.write(",".join(sorted(str(f) for f in family)) + '\n')

We generated an output file of these gene families, their median copy numbers, and number of members:

In [13]:
i=1
results=list()

for family in families_dedup:
    family_id="ID_"+str(i)
    family_mad=get_mad(family)
    family_members=str(len(family))
    family_proteins=len([element for element in family if "protein_coding" in element])
    family_pseudo=len([element for element in family if "unprocessed_pseudogene" in element])
    family_both=family_proteins+family_pseudo
    
    for f in family:
        f_wssd=wssd[wssd.index.isin([f])]
        f_wssd_median=f_wssd.median(axis=1)
        f_wssd_median_median=f_wssd_median.median()
        
        new_entry = {'gene_name': f, 'median_CN': f_wssd_median_median, 'family_id': family_id, 
                     "family_MAD": family_mad, "family_members": family_members, 
                     "family_proteins": family_proteins, "family_pseudo": family_pseudo, "family_both": family_both}
        results.append(new_entry)
    
    i+=1   

results = pd.DataFrame.from_records(results)
results.to_csv("results/SD98_families.tsv", sep="\t", index=False, na_rep="N/A")